In [2]:
#Import the necessary libraries 
import requests 
import json 
from env.api import openai_api_key
import openai

# imports
import pandas as pd
import numpy as np


from openai.embeddings_utils import cosine_similarity, get_embedding
from sklearn.metrics import PrecisionRecallDisplay

openai.api_key = openai_api_key

# parameters
EMBEDDING_MODEL = "text-embedding-ada-002"

In [3]:
def remove_newline(text):
    return text.replace('\n', '')

In [12]:
sample_schema="""
    {
    tables: dw_policies{
        columns:[incurred_loss, earned_prem, written_prem, earned_exp, claim_count, policy_id, quote_id, 
            effective_date,accident_date,calendar_date]
        keys: [policy_idD, quote_id]
        },
    dw_quotes{
        columns:[quote_id, partner_id, partner_name,state_abbrev]
        keys: [quote_id]
    }
    """

In [83]:


def generate_sql_preamble(schema:str,sql_type='snowflake'):
    table_desc= "Using only the following schema: \n" + schema + "\n" + "write a " + sql_type + "sql query that performs the following" 
    return table_desc

table_desc = generate_sql_preamble(sample_schema)

In [9]:
prompt="group by the month and year of accident date and state, and then sum premium and losses and calculate loss ratio"


In [38]:
prompt2="calculate the loss ratio for wisconsin in the calendar year of 2020"


In [85]:
def get_AI_sql_query(prompt:str,schema_str,model='text-davinci-003',temperature=.1,max_tokens=2000):
    
    total_prompt=generate_sql_preamble(schema_str) + prompt
    response=openai.Completion.create(prompt=total_prompt, model=model,temperature=temperature,max_tokens=max_tokens)
    sql_out=response['choices'][0]['text']
    return sql_out
    


In [13]:
print(get_AI_sql_query(prompt,sample_schema))



SELECT 
    DATE_TRUNC('month', dw_policies.accident_date) AS month_year,
    dw_quotes.state_abbrev,
    SUM(dw_policies.earned_prem) AS total_premium,
    SUM(dw_policies.incurred_loss) AS total_losses,
    SUM(dw_policies.incurred_loss) / SUM(dw_policies.earned_prem) AS loss_ratio
FROM dw_policies
INNER JOIN dw_quotes
    ON dw_policies.quote_id = dw_quotes.quote_id
GROUP BY
    DATE_TRUNC('month', dw_policies.accident_date),
    dw_quotes.state_abbrev;


In [87]:
def get_column_count(prompt,excess_columns=10, model='text-curie-001',temperature=.05,max_tokens=500):
    column_count_prompt="How many columns are needed for the following query? Respond with only a numerical character: "
    total_column_count_prompt=column_count_prompt+prompt

    response=openai.Completion.create(prompt=total_column_count_prompt, model=model,temperature=temperature,max_tokens=max_tokens)
    response_str=remove_newline(response['choices'][0]['text'])
    try:
        out=int(response_str)
        return out + excess_columns
    except ValueError:
        print("Warning: Could not estimate column count needed, defaulting to 20")
        return 20
    
 

In [15]:
get_column_count(prompt)

11

In [19]:

sample_schema2="""
Table 1: Customers
Columns: CustomerID, FirstName, LastName, Address, City, State, ZipCode

Table 2: Orders
Columns: OrderID, CustomerID, SalesPersonID, OrderDate, ShippingAddress, ShippingCity, ShippingState, ShippingZipCode 

Table 3: OrderItems 
Columns: OrderItemID, OrderID, ProductID, Quantity 

Table 4: Products 
Columns: ProductID, Name, Description, Price, Type, ManufacturingCost

Table 5: SalesPeople
Columns: SalesPersonID, FirstName LastName, Region, CommissionRate
"""

In [26]:
prompt3= """"
    sum the total profitability by product for all sales in wisconsin in 2022.  
    Include the cost of commission in the calculation of profitability
    """"
print(get_AI_sql_query(prompt3,schema_str=sample_schema2))



SELECT p.Name, SUM(oi.Quantity * (p.Price - p.ManufacturingCost - (p.Price * sp.CommissionRate))) AS TotalProfitability
FROM Products p
JOIN OrderItems oi ON oi.ProductID = p.ProductID
JOIN Orders o ON o.OrderID = oi.OrderID
JOIN SalesPeople sp ON sp.SalesPersonID = o.SalesPersonID
JOIN Customers c ON c.CustomerID = o.CustomerID
WHERE YEAR(o.OrderDate) = 2022 AND c.State = 'Wisconsin'
GROUP BY p.Name;


In [65]:
def knn_cosine_similarity(query:str,documents,document_embeddings, k):
    query_embedding=get_embedding(query, engine=EMBEDDING_MODEL)
    similarities = []
    for document, embedding in zip(documents,document_embeddings):
        sim_iter=cosine_similarity(query_embedding,embedding)
        similarities.append((document, sim_iter))

    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True) 

    best_labels = [x[0] for x in sorted_similarities[:k]] 
    similarities = [x[1] for x in sorted_similarities[:k]] 

    return best_labels

In [27]:
sample_schema_dict = {
    'Customers': {
        'columns': ['CustomerID', 'FirstName', 'LastName', 'Address', 'City', 'State', 'ZipCode']
    },
    'Orders': {
        'columns': ['OrderID', 'CustomerID', 'SalesPersonID', 'OrderDate', 
                    'ShippingAddress', 'ShippingCity', 'ShippingState','ShippingZipCode']
    }, 
    'OrderItems': {
        'columns': ['OrderItemID','OrderID','ProductID','Quantity']
    }, 
    'Products': { 
        'columns': ['ProductID','Name','Description','Price','ProductTypeID','ManufacturingCost','ColorID','ShippingDimX','ShippingDimY','ShippingDimZ','ShippingWeight'] 
    }, 
    "SalesPeople": { 
        "columns": ["SalesPersonID", "FirstName", "LastName", "Region", "CommissionRate"] 
     },
    'ProductTypes': { 
        'columns': ['ProductTypeID','ProductTypeDescription','ManufacturingPlantID'] 
    },
    'ManufacturingPlants': { 
        'columns': ['ManufacturingPlantID','Country',"StateRegion"] 
    }
}

In [53]:
def flatten_schema(schema_dict:dict):
    data_out=pd.DataFrame(columns=['table','column'])
    for table, columns in schema_dict.items():
        for column in columns['columns']: ##need to make this more robust
            data_out=pd.concat([data_out,pd.DataFrame( data={'table':table,'column':column},index=[0])])
    return data_out



In [55]:
schema_data=flatten_schema(sample_schema_dict)

schema_data['embeddings'] = schema_data.apply(lambda x: get_embedding(x['column'],engine=EMBEDDING_MODEL) , axis=1)

In [62]:
schema_labels_list=(schema_data['table']+ ':' + schema_data['column']).to_list()

k=get_column_count(prompt3)
knn_cosine_similarity(prompt3,documents=schema_labels_list,document_embeddings=schema_data['embeddings'],k=k)

Could not convert string to integer.


(['Products:ManufacturingCost',
  'Products:Price',
  'SalesPeople:CommissionRate',
  'OrderItems:Quantity',
  'OrderItems:ProductID',
  'Products:ProductID',
  'Orders:SalesPersonID',
  'SalesPeople:SalesPersonID',
  'Products:ShippingWeight',
  'ProductTypes:ProductTypeDescription',
  'Customers:State',
  'Orders:CustomerID',
  'Customers:CustomerID',
  'SalesPeople:Region',
  'Products:ShippingDimZ'],
 [0.7947047602896089,
  0.7901440959178926,
  0.765955064742226,
  0.7645188042884107,
  0.7614922740164835,
  0.7614922740164835,
  0.7611632894064284,
  0.761149176654435,
  0.759727795420023,
  0.7556596847497794,
  0.7546947558375547,
  0.7542914105006838,
  0.7542804780908171,
  0.7521845072981967,
  0.7518444457610142])

In [63]:
id_col_prompt="ID column, key or identifier"
knn_cosine_similarity(id_col_prompt,documents=schema_labels_list,document_embeddings=schema_data['embeddings'],k=15)

(['Products:ColorID',
  'Orders:CustomerID',
  'Customers:CustomerID',
  'OrderItems:ProductID',
  'Products:ProductID',
  'OrderItems:OrderID',
  'Orders:OrderID',
  'SalesPeople:SalesPersonID',
  'Orders:SalesPersonID',
  'Products:ProductTypeID',
  'ProductTypes:ProductTypeID',
  'OrderItems:OrderItemID',
  'Products:Name',
  'ProductTypes:ManufacturingPlantID',
  'ManufacturingPlants:ManufacturingPlantID'],
 [0.8392546286171951,
  0.8173117715855429,
  0.8172743121043944,
  0.8152707950045406,
  0.8152707950045406,
  0.8129136976760638,
  0.8128540464313366,
  0.8062273414133921,
  0.8061629454786857,
  0.8013942899803265,
  0.8013942899803265,
  0.798274964391725,
  0.7859698386694702,
  0.7831799801511035,
  0.7831799801511035])

In [88]:
def get_schema_subset_str(prompt,schema_data:pd.DataFrame):
    k=get_column_count(prompt)
    schema_labels_list=(schema_data['table']+ ':' + schema_data['column']).to_list()
    column_data=knn_cosine_similarity(prompt,documents=schema_labels_list,document_embeddings=schema_data['embeddings'],k=k)
    
    id_col_prompt="ID columns, keys or identifiers related to the following prompt: " + prompt
    key_data= knn_cosine_similarity(id_col_prompt,documents=schema_labels_list,document_embeddings=schema_data['embeddings'],k=15)
    delimiter=', '
    schema_text='Schema Format is (Table:Column) \n' + delimiter.join(column_data) + delimiter.join(key_data)
    return schema_text



In [89]:
prompt4=""" 
    What is the total revenue sold by Jerry in 2023?  And how much total commission did we pay him?
    
    """

schema_subset_str= get_schema_subset_str(prompt4,schema_data)

print(get_AI_sql_query(prompt4,schema_subset_str))


SELECT SUM(Products.Price * OrderItems.Quantity) AS Total_Revenue, 
       SUM(Products.Price * OrderItems.Quantity * SalesPeople.CommissionRate) AS Total_Commission
FROM Products 
INNER JOIN OrderItems ON Products.ProductID = OrderItems.ProductID
INNER JOIN Orders ON OrderItems.OrderID = Orders.OrderID
INNER JOIN SalesPeople ON Orders.SalesPersonID = SalesPeople.SalesPersonID
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
WHERE SalesPeople.FirstName = 'Jerry' 
AND YEAR(Orders.OrderDate) = 2023;


In [90]:
prompt4=""" 
    How many 'Johnson Rods' did we sell during the fourth finanical quarter of 2022 in Wisconsin?
    
    """

schema_subset_str= get_schema_subset_str(prompt4,schema_data)

print(get_AI_sql_query(prompt4,schema_subset_str))


SELECT SUM(OrderItems.Quantity)
FROM OrderItems
INNER JOIN Products ON OrderItems.ProductID = Products.ProductID
INNER JOIN Orders ON OrderItems.OrderID = Orders.OrderID
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
WHERE Products.Description = 'Johnson Rods'
AND Customers.State = 'Wisconsin'
AND Orders.OrderDate BETWEEN '2022-10-01' AND '2022-12-31';


In [91]:
prompt4=""" 
    How much commission did we pay Newman in 2022 to sell 'Johnson Rods'?  
    
    """

schema_subset_str= get_schema_subset_str(prompt4,schema_data)

print(get_AI_sql_query(prompt4,schema_subset_str))


SELECT SUM(OrderItems.Quantity * Products.Price * SalesPeople.CommissionRate) AS TotalCommission
FROM OrderItems
INNER JOIN Products ON OrderItems.ProductID = Products.ProductID
INNER JOIN Orders ON OrderItems.OrderID = Orders.OrderID
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
INNER JOIN SalesPeople ON Orders.SalesPersonID = SalesPeople.SalesPersonID
WHERE SalesPeople.FirstName = 'Newman'
AND Products.Description = 'Johnson Rods'
AND YEAR(Orders.OrderDate) = 2022;
